# Unsupervised Domain Adaptation Project


## Part-1: Data download
Load data to project from Google Drive. Copy a subset of classes of images to the path:
- `adaptiope_small/product_images`
- `adaptiope_small/real_life` 

two directories. They represent images from two different domain **product** and **real_life**

In [1]:
from os import makedirs, listdir
from tqdm import tqdm
from google.colab import drive
from os.path import join
from shutil import copytree

drive.mount('/content/gdrive')

!mkdir dataset
!cp "gdrive/My Drive/Colab Notebooks/data/Adaptiope.zip" dataset/
# !ls dataset

!unzip -qq dataset/Adaptiope.zip   # unzip file

!rm -rf dataset/Adaptiope.zip 
!rm -rf adaptiope_small

Mounted at /content/gdrive


In [2]:
!mkdir adaptiope_small
classes = listdir("Adaptiope/product_images")
print(classes)
classes = ["backpack", "bookcase", "car jack", "comb", "crown", "file cabinet", "flat iron", "game controller", "glasses",
           "helicopter", "ice skates", "letter tray", "monitor", "mug", "network switch", "over-ear headphones", "pen",
           "purse", "stand mixer", "stroller"]
domain_classes = ["product_images", "real_life"]
for d, td in zip(["Adaptiope/product_images", "Adaptiope/real_life"], ["adaptiope_small/product_images", "adaptiope_small/real_life"]):
  makedirs(td)
  for c in tqdm(classes):
    c_path = join(d, c)
    c_target = join(td, c)
    copytree(c_path, c_target)

['magic lamp', 'car jack', 'bicycle', 'hat', 'spatula', 'brachiosaurus', 'rubber boat', 'microwave', 'smoking pipe', 'toothbrush', 'quadcopter', 'crown', 'scissors', 'ice skates', 'mug', 'file cabinet', 'pipe wrench', 'keyboard', 'projector', 'fan', 'usb stick', 'handcuffs', 'glasses', 'hoverboard', 'backpack', 'computer', 'vacuum cleaner', 'stapler', 'notepad', 'hand mixer', 'handgun', 'rc car', 'coat hanger', 'office chair', 'hot glue gun', 'electric shaver', 'game controller', 'scooter', 'hourglass', 'nail clipper', 'comb', 'dart', 'calculator', 'chainsaw', 'binoculars', 'umbrella', 'acoustic guitar', 'helicopter', 'lawn mower', 'monitor', 'rifle', 'tyrannosaurus', 'razor', 'screwdriver', 'stroller', 'printer', 'webcam', 'golf club', 'pen', 'ruler', 'axe', 'telescope', 'fire extinguisher', 'phonograph', 'knife', 'wristwatch', 'diving fins', 'cellphone', 'syringe', 'fighter jet', 'desk lamp', 'skateboard', 'over-ear headphones', 'bicycle helmet', 'drum set', 'vr goggles', 'wheelchair

100%|██████████| 20/20 [00:01<00:00, 18.34it/s]


## Part-2: Image Classification Neural Network

 

### Part-2.0: Data Loading

First we load the data and preprocessing them

In [3]:
product_path = 'adaptiope_small/product_images'
real_life_path = 'adaptiope_small/real_life'

In [4]:
!pwd
!ls

/content
Adaptiope  adaptiope_small  dataset  gdrive  sample_data


In [5]:
from PIL import Image
from os.path import join

img = Image.open(join(product_path, 'backpack', 'backpack_003.jpg'))
print('Image size: ', img.size)
#img

Image size:  (679, 679)


import libraries

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import vgg11, alexnet 
from torch.utils.data import DataLoader, random_split

configuration constants

In [7]:
img_size = 256
# mean, std used by pre-trained models from PyTorch
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
config = dict(epochs=5, batch_size=64,lr=0.001, wd=0.001, momentum=0.9, domain_regression_weight=0.3)

Configue GPU

In [8]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [9]:
from torchvision.transforms.transforms import ToTensor

def get_dataset(root_path):
  '''
    Get dataset from specific data path

    # parameters:
        root_path: path to image folder

    # return: train_loader, test_loader
  '''
  # Construct image transform
  image_transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
  ])

  # Load data from filesystem
  image_dataset = ImageFolder(root_path, transform=image_transform)

  return image_dataset

def get_dataloader(dataset, batch_size, shuffle_train=True, shuffle_test=False):
  '''
    Get DataLoader from specific data path

    # parameters:
        dataset: ImageFolder instance
        batch_size: batch_size for DataLoader
        shuffle_train: whether to shuffle training data
        shuffle_test: whether to shuffle test data
  '''
  # Get train, test number
  num_total = len(dataset)
  num_train = int(num_total * 0.8 + 1)
  num_test  = num_total - num_train

  # random split dataset
  data_train, data_test = random_split(dataset, [num_train, num_test])

  # initialize dataloaders
  loader_train = DataLoader(data_train, batch_size=batch_size, shuffle=shuffle_train)
  loader_test  = DataLoader(data_test, batch_size=batch_size, shuffle=shuffle_test)

  return loader_train, loader_test

### Part-2.1 Pretrain Network

Here we use a pretrain Neural Network to start with, then we fine tune it with the data set we have from **Adaptiope** in one domain, and test it on the target domain. Compare the two result, and set the benchmark for later UDA enriched method. 

In [10]:
# pd_dataset = get_dataset(product_path)
# len(pd_dataset.classes)

### Part-2.2 Define the Model with Feature Extractor, Classifier, and Domain Regressor.

In [12]:
class FeatureExtractor(nn.Module):
  def __init__(self):
    super(FeatureExtractor, self).__init__()

    # Feature Extractor with AlexNet
    self.feature_extractor = alexnet(pretrained=True)
    self.feature_dim = self.feature_extractor.classifier[-1].in_features

    # make the last layer identity
    self.feature_extractor.classifier[-1] = nn.Identity()

  def forward(self, x):
    return self.feature_extractor(x)
  
  def output_dim(self):
    return self.feature_dim

In [13]:
from torch import softmax


class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, output_dim)
        )
    
    def forward(self, X):
        return self.classifier(X) 

In [14]:
from torch.autograd import Function

class ReverseLayerF(Function):
    @staticmethod
    def forward(ctx, tensor):
        return tensor.view_as(tensor)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg(), None

In [15]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.discriminator =  nn.Sequential(
            nn.Linear(int(input_dim), 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024,1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        validity = self.discriminator(x)
        return validity 

In [16]:
class DANN(nn.Module):
  # def __init__(self, num_classes, adversarial=True):
  def __init__(self, num_classes):
    super(DANN, self).__init__()
    self.output_dim = num_classes

    # define inner network component
    self.feature_extractor = FeatureExtractor()
    self.classifier = Classifier(self.feature_extractor.output_dim(), num_classes)
    self.discriminator = Discriminator(self.feature_extractor.output_dim())  
  
  def forward(self, x):
    feature_output = self.feature_extractor(x)

    class_pred = self.classifier(feature_output)

    # Add a ReverseLayer here for negative gradient computation
    reverse_feature = ReverseLayerF.apply(feature_output)
    domain_pred = self.discriminator(reverse_feature)

    return class_pred, domain_pred 

### Part-2.2.1 Define the Feature Extractor by using a pretrain model

### Part-2.2.2 Define the Classifier

### Part-2.2.3 Define the Domain Regressor

### Part-2.3 Cost function

Divide parameters intro two groups, in which the last fully conneted layer with learning_rate, the other layers with 0.1 * learning_rate.

In [20]:
def get_class_loss_func():
  return nn.CrossEntropyLoss()

# def get_domain_loss_func():
  # return torch.nn.BCELoss()

### Part-2.4 Optimizer

In [22]:
def get_optimizer(model, config, adversarial=True):
  '''
  Config Optimizer
  '''
  learning_rate = config['lr']
  weight_decay  = config['wd']
  momentum      = config['momentum']

  feature_ext   = model.get_submodule("feature_extractor")
  classifier    = model.get_submodule("classifier")
  discriminator = model.get_submodule("discriminator")

  # classification layer name: label_classifier
  # classifier_name = "label_classifier"
  # domain_regressor_name = "domain_regressor"
  pre_trained_weights = feature_ext.parameters()

  if adversarial:
    other_weights = list(classifier.parameters()) + list(discriminator.parameters())
  else:
    other_weights = list(classifier.parameters())

  # assign parameters to parameters
  optimizer = torch.optim.SGD([
    {'params': pre_trained_weights},
    {'params': other_weights, 'lr': learning_rate}
  ], lr= learning_rate/10, weight_decay=weight_decay, momentum=momentum)
  
  return optimizer

### Part-2.5 Training and Testing Step

In [23]:
def train_loop(dataloader, model, optimizer, device):
  size = len(dataloader.dataset)
  loss_fn = get_class_loss_func()

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    
    # compute prediction and loss
    class_pred, _ = model(X)

    # classification loss
    loss = loss_fn(class_pred, y)

    # backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
    
    del loss

In [24]:
def test_loop(dataloader, model, device):
  test_loss, correct = 0, 0
  loss_fn = get_class_loss_func()

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      class_pred, _ = model(X)

      test_loss += loss_fn(class_pred, y).item()
      correct += (class_pred.argmax(1) == y).type(torch.float).sum().item()

  size = len(dataloader.dataset)
  num_batches = len(dataloader)

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

  return test_loss, correct

### Part-2.6 Training & Testing on Target Domain

In [25]:
def training(model, train_dataloader, test_dataloader, config, device):
  epochs = config['epochs']
  print(f"Learning_rate {config['lr']}, weight_decay {config['wd']}")

  optimizer = get_optimizer(model, config, adversarial=False)

  for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n------------------")
    train_loop(train_dataloader, model, optimizer, device)
    test_loop(test_dataloader, model, device)

  print("Done")

In [26]:
# Get dataloader
product_dataset = get_dataset(product_path)
real_life_dataset = get_dataset(real_life_path)

### Product -> Real Life
#### Train on Product

In [27]:
train_dataloader, test_dataloader = get_dataloader(product_dataset, 64)

model = DANN(len(product_dataset.classes)).to(device)

# Training
training(model, train_dataloader, test_dataloader, config, device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

Learning_rate 0.001, weight_decay 0.001
Epoch 1
------------------
loss: 3.026970 [    0/ 1601]
Test Error: 
 Accuracy: 61.2%, Avg loss: 2.707559 

Epoch 2
------------------
loss: 2.640605 [    0/ 1601]
Test Error: 
 Accuracy: 78.7%, Avg loss: 2.089261 

Epoch 3
------------------
loss: 1.968558 [    0/ 1601]
Test Error: 
 Accuracy: 83.2%, Avg loss: 1.018701 

Epoch 4
------------------
loss: 1.043317 [    0/ 1601]
Test Error: 
 Accuracy: 85.7%, Avg loss: 0.538009 

Epoch 5
------------------
loss: 0.339218 [    0/ 1601]
Test Error: 
 Accuracy: 85.7%, Avg loss: 0.422970 

Done


#### Test on Real Life

In [28]:
loader_target_dataset = DataLoader(real_life_dataset, batch_size=64, shuffle=False)

# model.load_state_dict(torch.load('model_state.pt', map_location='cpu'))
test_loop(loader_target_dataset, model, device)

Test Error: 
 Accuracy: 62.5%, Avg loss: 1.437788 



(1.437787752598524, 0.6245)

In [29]:
del train_dataloader, test_dataloader, loader_target_dataset
del model
print(torch.cuda.memory_allocated())

513480192


### Real Life -> Product
#### Train on Real Life

In [30]:
train_dataloader, test_dataloader = get_dataloader(real_life_dataset, 64)

model = DANN(len(real_life_dataset.classes)).to(device)

# Training
training(model, train_dataloader, test_dataloader, config, device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Learning_rate 0.001, weight_decay 0.001
Epoch 1
------------------
loss: 2.987454 [    0/ 1601]
Test Error: 
 Accuracy: 21.8%, Avg loss: 2.887071 

Epoch 2
------------------
loss: 2.871017 [    0/ 1601]
Test Error: 
 Accuracy: 45.6%, Avg loss: 2.698080 

Epoch 3
------------------
loss: 2.674101 [    0/ 1601]
Test Error: 
 Accuracy: 59.9%, Avg loss: 2.408932 

Epoch 4
------------------
loss: 2.469381 [    0/ 1601]
Test Error: 
 Accuracy: 64.2%, Avg loss: 1.896553 

Epoch 5
------------------
loss: 1.808306 [    0/ 1601]
Test Error: 
 Accuracy: 75.2%, Avg loss: 1.272889 

Done


#### Test on Product


In [31]:
loader_target_dataset = DataLoader(product_dataset, batch_size=64, shuffle=False)

# model.load_state_dict(torch.load('model_state.pt', map_location='cpu'))
test_loop(loader_target_dataset, model, device)

Test Error: 
 Accuracy: 75.2%, Avg loss: 1.180410 



(1.1804103730246425, 0.752)

In [32]:
del train_dataloader, test_dataloader, loader_target_dataset
del model
print(torch.cuda.memory_allocated())

513021440


## TODO

TODO: Dataset unzip Google Drive, Copy to folder

TODO: Batch progress number error

TODO: Early stop or dropout, when accuracy doesn't improve much

Otherwise Continue UDA

## 3: UDA 


### 3.1 Adversarial Discriminator 

Discriminator Loss

In [107]:
def get_discriminator_loss(source_pred, target_pred): 
    domain_pred = torch.cat((source_pred, target_pred),dim=0).cuda()
    #print(domain_pred.shape) # [128,1024]
    source_truth = torch.zeros(len(source_pred))
    target_truth = torch.ones(len(target_pred))
    domain_truth = torch.cat((source_truth, target_truth),dim=0).cuda()
    #print(domain_truth.shape) # [128]

    domain_loss = domain_truth*torch.log(1/domain_pred)+(1-domain_truth)*torch.log(1/(1-domain_pred))
    domain_loss = domain_loss.mean()

    return domain_loss 

Classification Loss

### 3.2 Adversarial optimizer

In [38]:
def get_adversarial_optimizer(model, config, adversarial=True):
  '''
  Get Adversarial Optimizers
  '''
  lr, wd, momtm = config['lr'], config['wd'], config['momentum']

  feature_ext   = model.get_submodule("feature_extractor")
  classifier    = model.get_submodule("classifier")
  discriminator = model.get_submodule("discriminator")

  pre_trained_weights   = feature_ext.parameters()
  classifier_weights    = classifier.parameters()
  discriminator_weights = discriminator.parameters()

  feature_optim       = torch.optim.SGD([{'params': pre_trained_weights}],     lr=lr/10, weight_decay=wd, momentum=momtm)
  classifier_optim    = torch.optim.SGD([{'params': classifier_weights}],      lr=lr,    weight_decay=wd, momentum=momtm)
  discriminator_optim = torch.optim.SGD([{'params': discriminator_weights}],   lr=lr,    weight_decay=wd, momentum=momtm)
  
  return feature_optim, classifier_optim, discriminator_optim 

### 3.3 Domain Regression Loss

### 3.4 Adversarial Train Loop

In [101]:
def adversarial_train_loop(source_loader, target_loader, model, config, device):
  """
  domain_adapt: lambda in the objective function
  """
  size = len(source_loader.dataset)
  domain_weight = config['domain_regression_weight']
  
  # cross entropy loss
  classification_loss = get_class_loss_func()

  # Get three optimizer
  feature_optim, class_optim, discriminator_optim = get_adversarial_optimizer(model, config)

  # Target data loader iterator
  iter_target = iter(target_loader)

  # TODO: need to use formula
  domain_adapt = 1

  for batch, (X_source, y_source) in enumerate(source_loader):
    try:
      X_target, _ = next(iter_target)
    except:
      iter_target = iter(target_loader)
      X_target, _ = next(iter_target)
    
    

    # Some internal bug return nested tesnor with size 1
    if len(X_source) < 64:
      continue
    
    # print("Len source: ", len(X_source), " Len target: ", len(X_target))
    # if len(X_source) < 64: print(X_source, " ", X_target)

    X_source, y_source, X_target = X_source.to(device), y_source.to(device), X_target.to(device)

    class_pred_source, domain_pred_source = model(X_source)
    _,                 domain_pred_target = model(X_target)

    # print("class_pred_source")
    # print(class_pred_source[1])
    # print("domain_pred_source")
    # print(domain_pred_source[1])
    # print("domain_pred_target")
    # print(domain_pred_target[1])

    # if batch % 2 == 0:
      # Train classifier 
    class_loss   = classification_loss(class_pred_source, y_source)
    discrim_loss = get_discriminator_loss(domain_pred_source, domain_pred_target)


    feature_optim.zero_grad()

    # Update discriminator
    discriminator_optim.zero_grad()
    discrim_loss.backward(retain_graph=True)
    discriminator_optim.step()

    # Update classifier
    class_optim.zero_grad()
    class_loss.backward(retain_graph=True)
    class_optim.step()

    # Update feature extractor
    feature_optim.step()  

    # Total loss
    total_loss = class_loss - domain_adapt * discrim_loss 
    """
    # Update classifier and feature extractor
    total_loss = class_loss - domain_adapt * discrim_loss 

    class_optim.zero_grad()
    feature_optim.zero_grad()
    # discriminator_optim.zero_grad()

    total_loss.backward()

    class_optim.step()
    feature_optim.step()
    # discriminator_optim.step()
    """

    if batch % 10 == 0:
      class_loss, discrim_loss, current = class_loss.item(), discrim_loss.item(), batch * len(X_source)
      total_loss = total_loss.item()
      print(f"classification loss: {class_loss:>7f} discrim loss: {discrim_loss:>7f} total loss: {total_loss:>7f}[{current:>5d}/{size:>5d}]")


    del class_loss, discrim_loss 
    del X_source, y_source, X_target, class_pred_source, domain_pred_source, domain_pred_target

#### Total Loss = Classification Loss + Discrimination Loss

### 3.5 Adversarial Test Loop

In [43]:
def adversarial_test_loop(dataloader, model, device, name=""):
  test_loss, correct = 0, 0

  class_loss_func = get_class_loss_func()

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      class_pred, _ = model(X)

      test_loss += class_loss_func(class_pred, y).item()
      correct += (class_pred.argmax(1) == y).type(torch.float).sum().item()

  size = len(dataloader.dataset)
  num_batches = len(dataloader)

  test_loss /= num_batches
  correct /= size
  print(f"{name} Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

  return test_loss, correct

### 3.6 Adversarial training

In [44]:
def adversarial_training(model, source_loader, source_test_loader, target_loader, config, device):
  print(f"Learning_rate {config['lr']}, weight_decay {config['wd']}")

  for epoch in range(config['epochs']):
    print(f"Epoch {epoch+1}\n------------------")
    adversarial_train_loop(source_loader, target_loader, model, config, device)
    adversarial_test_loop(source_test_loader, model, device, "Source Test")
    adversarial_test_loop(target_loader, model, device, "Target Train")

  print("Done")

### 3.7 Adversarial Testing

### Product ->  Real Life

In [108]:
torch.cuda.empty_cache()

In [109]:
adv_model = DANN(len(product_dataset.classes)).to(device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [110]:
train_dataloader, train_test_dataloader = get_dataloader(product_dataset, 64)
target_dataloader, target_test_dataloader = get_dataloader(real_life_dataset, 64)

In [111]:
torch.autograd.set_detect_anomaly(True)
config['epochs'] = 10
adversarial_training(adv_model, train_dataloader, train_test_dataloader, target_dataloader, config, device)

Learning_rate 0.001, weight_decay 0.001
Epoch 1
------------------
classification loss: 2.961929 discrim loss: 0.698104 total loss: 2.263826[    0/ 1601]
classification loss: 2.945867 discrim loss: 0.695418 total loss: 2.250449[  640/ 1601]
classification loss: 2.794593 discrim loss: 0.693935 total loss: 2.100658[ 1280/ 1601]
Source Test Test Error: 
 Accuracy: 47.6%, Avg loss: 2.727703 

Target Train Test Error: 
 Accuracy: 28.0%, Avg loss: 2.889599 

Epoch 2
------------------
classification loss: 2.747331 discrim loss: 0.694111 total loss: 2.053219[    0/ 1601]
classification loss: 2.628558 discrim loss: 0.694059 total loss: 1.934499[  640/ 1601]
classification loss: 2.373319 discrim loss: 0.694176 total loss: 1.679143[ 1280/ 1601]
Source Test Test Error: 
 Accuracy: 67.7%, Avg loss: 2.323300 

Target Train Test Error: 
 Accuracy: 44.3%, Avg loss: 2.720714 

Epoch 3
------------------
classification loss: 2.294928 discrim loss: 0.694141 total loss: 1.600787[    0/ 1601]
classificati

In [112]:
loader_target_dataset = DataLoader(real_life_dataset, batch_size=64, shuffle=False)

test_loop(loader_target_dataset, adv_model, device)

Test Error: 
 Accuracy: 63.0%, Avg loss: 1.355239 



(1.3552394360303879, 0.6305)

#### Real Life -> Product

In [113]:
train_dataloader, train_test_dataloader = get_dataloader(real_life_dataset, 64)
target_dataloader, target_test_dataloader = get_dataloader(product_dataset, 64)

In [114]:
# Training
config['epochs'] = 10
adversarial_training(adv_model, train_dataloader, train_test_dataloader, target_dataloader, config, device)

Learning_rate 0.001, weight_decay 0.001
Epoch 1
------------------
classification loss: 1.225204 discrim loss: 0.695646 total loss: 0.529557[    0/ 1601]
classification loss: 1.199243 discrim loss: 0.695601 total loss: 0.503642[  640/ 1601]
classification loss: 1.095573 discrim loss: 0.695219 total loss: 0.400354[ 1280/ 1601]
Source Test Test Error: 
 Accuracy: 66.7%, Avg loss: 1.220216 

Target Train Test Error: 
 Accuracy: 90.5%, Avg loss: 0.290472 

Epoch 2
------------------
classification loss: 0.952599 discrim loss: 0.695886 total loss: 0.256713[    0/ 1601]
classification loss: 0.907439 discrim loss: 0.696670 total loss: 0.210769[  640/ 1601]
classification loss: 0.873558 discrim loss: 0.696532 total loss: 0.177026[ 1280/ 1601]
Source Test Test Error: 
 Accuracy: 69.4%, Avg loss: 1.049284 

Target Train Test Error: 
 Accuracy: 90.5%, Avg loss: 0.272123 

Epoch 3
------------------
classification loss: 0.972146 discrim loss: 0.695485 total loss: 0.276662[    0/ 1601]
classificati

In [115]:
loader_target_dataset = DataLoader(real_life_dataset, batch_size=64, shuffle=False)

test_loop(loader_target_dataset, adv_model, device)

Test Error: 
 Accuracy: 86.6%, Avg loss: 0.440376 



(0.4403761252760887, 0.8655)

### 3.8 Testing on Target Domain

In [106]:
# del source_dataset, train_dataloader, test_dataloader, target_dataset, loader_target_dataset
del adv_model

## Part-4: Comparison & Discussion
Here we compare the test result from the direct method and the UDA method. 

## Part-5: Conclusion